# Amazon SageMaker Demo
## 使用 XGBoost 预测移动用户流失
---
## 内容
1. [背景](#背景)
2. [准备环境](#准备环境)
3. [准备数据](#准备数据)
4. [模型训练](#模型训练)
5. [模型编译](#模型编译)
6. [模型评估](#模型托管)
7. [（可选）清理环境](#（可选）清理环境)
---
## 背景

这个 Demo 改编自该博客：
[AWS blog post](https://aws.amazon.com/blogs/ai/predicting-customer-churn-with-amazon-machine-learning/)

失去客户对于任何业务来说都是代价高昂的。
尽早识别不满意的客户可以让您有机会为他们提供挽留奖励。
在这个通过使用机器学习自动识别不满意的客户，也称为客户流失预测。
并且通过使用 Amazon SageMaker 功能来管理实验、训练和调试模型，并在部署后对其进行监控。
---
## 准备环境

使用之前需要指定:
- Demo 中需要用到的 Python 库
- S3 存储桶和前缀用于存放训练数据和模型数据
- IAM 角色允许训练和托管时可以访问到相应的数据

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import datetime
import json
from IPython.display import display
from time import strftime, gmtime
import sagemaker
from sagemaker.predictor import csv_serializer

In [ ]:
# Define S3 bucket
current_time = datetime.datetime.now().strftime('%G%m%d-%H%M%S')
bucket = 'sagemaker-bjs-ryanlao'
prefix = 'demo/xgb/customer-churn-' + current_time

# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()
# role = 'arn:aws-cn:iam::855501529395:role/SAGEMAKER_RW_ALL'

---
## 准备数据

移动运营商有历史记录来体现他们哪些客户最终流失，哪些客户继续使用该服务。
我们可以使用这些历史信息来训练可以预测客户流失的机器学习模型。
训练模型后，我们可以将任意客户的配置文件信息（与我们用于训练模型的配置文件信息相同）传递给模型，以便模型预测该客户是否会流失。

In [ ]:
!apt-get -y -qq install unzip
!wget https://sagemaker-lab-zhy-ryanlao.s3.cn-northwest-1.amazonaws.com.cn/dataset/DKD2e_data_sets.zip
!unzip -o DKD2e_data_sets.zip

In [ ]:
# Load raw data
churn = pd.read_csv('./Data sets/churn.txt')
pd.set_option('display.max_columns', 500)
churn

按照现代标准，它是一个相对较小的数据集，只有 3,333 条记录，其中每条记录使用 21 个属性来描述一个未知的美国移动运营商的客户档案。这些属性包括：

- `State`: 美国的州缩写
- `Account Length`: 帐户处于活动状态的天数
- `Area Code`: 客户电话号码的三位数区号
- `Phone`: 客户电话号码
- `Int’l Plan`: 客户是否使用国际漫游(yes/no)
- `VMail Plan`: 客户是否使用语音信箱(yes/no)
- `VMail Message`: 每月平均语音邮件数量
- `Day Mins`: 白天的呼叫时长
- `Day Calls`: 白天的呼叫次数
- `Day Charge`: 白天的呼叫费用
- `Eve Mins, Eve Calls, Eve Charge`: 晚上拨打电话的计费费用
- `Night Mins`, `Night Calls`, `Night Charge`: 夜间拨打电话的计费费用
- `Intl Mins`, `Intl Calls`, `Intl Charge`: 国际通话的计费费用
- `CustServ Calls`: 打客服电话的次数
- `Churn?`: 客户是否流失(true/false)

最后一个属性 `Churn?` 是目标属性，即我们希望机器学习模型预测的属性。
由于目标属性是二进制的，因此我们的模型将执行二进制预测，也称为二进制分类（二分类）。


In [ ]:
# 使用交叉表(crosstab)显示每个类别特征的频率
for column in churn.select_dtypes(include=['object']).columns:
    display(pd.crosstab(index=churn[column],
                        columns='% observations',
                        normalize='columns'))

# 每个数值特征的直方图(hist)
print("count:\t总数量\n", "mean:\t均值\n", "std:\t标准差\n", "min:\t最小值\n", "%:\t较低百分数位，中间百分数位，较高百分数位\n", "max:\t最大值")
display(churn.describe())
%matplotlib inline
hist = churn.hist(bins=30, sharey=True, figsize=(10, 10))

可以观察到：
- `State` 相当均匀的分布
- `Phone` 均是唯一值，如果能解析出前缀可能会有一些价值，但是如果没有上下文关联的话，可以避免使用
- 只有 14% 的客户流失，数据分类不太均衡，但并不极端
- 大多数特征都非常好地呈现出高斯分布。
- `VMail Message` 是一个明显的例外 (`Area Code` 看上去需要转化成非数字化)

In [ ]:
# axis = 1 for dropping cplumn index as well
churn = churn.drop('Phone', axis=1)
# Change 'Area Code' type to object
churn['Area Code'] = churn['Area Code'].astype(object)

接下来看看每个特征和我们的目标变量之间的关系。

In [ ]:
for column in churn.select_dtypes(include=['object']).columns:
    if column != 'Churn?':
        display(pd.crosstab(index=churn[column],
                            columns=churn['Churn?'],
                            normalize='columns'))

for column in churn.select_dtypes(exclude=['object']).columns:
    print(column)
    hist = churn[[column, 'Churn?']].hist(by='Churn?', bins=30)
    plt.show()

有趣的是，我们看到流失的客户出现的特征是:
- 地理分布相当均匀
- 开通国际漫游的客户更有可能会流失
- 不太可能开启语音信箱
- 在白天通话呈现出两种情况 (明显高于或者低于未流失客户的平均值)
- 更多次的客服通话 (这也是比较合理的，可以预计遇到大量问题的客户可能更容易流失)

此外也可以看到，流失的客户在白天通话时长和白天通话费用这两个特征上显示出非常相似的分布。这并不奇怪，因为通话时长和计费也是相关联的.

In [ ]:
# 通过散布矩阵查看数据关联和规律
display(churn.corr())
pd.plotting.scatter_matrix(churn, figsize=(12, 12))
plt.show()

我们看到几个特征本质上彼此之间具有 100% 的相关性。在某些机器学习算法中如果包含这些特征对可能会造成灾难性问题，而在另一些算法中，它只会引入少量冗余和偏差。现在先把如下的计费项移除，因为他们和通话时长高度相关：
- 白天通话计费与白天通话时长相对
- 晚上通话计费与晚上通话时长相对
- 夜间通话计费与夜间通话时长相对
- 国际漫游计费和国际漫游时长相对

In [ ]:
churn = churn.drop(['Day Charge',
                    'Eve Charge',
                    'Night Charge',
                    'Intl Charge'],
                   axis=1)

规整了数据集，现在要确定使用哪种算法。
如上所述，似乎有一些变量，其中高值和低值（但不是中间值）都可以预测流失。
通过使用Amazon SageMaker提供的XGBoost算法容器，可以在托管的分布式设置中进行训练，然后将其作为实时预测终端进行托管。
XGBoost使用渐变增强树，这些树自然地考虑到各个特征和目标特征之间的非线性关系，以及各特征之间的复杂交互。

XGBoost 可以对 CSV 和 LibSVN 格式的数据进行训练，当前Demo环境的数据格式为CSV:
- 第一列为预测特征
- 没有标题行

先将类别特征转换为数值特征。

In [ ]:
# Encode as one hot
model_data = pd.get_dummies(churn)

# concat 可以将数据根据不同的轴作简单的融合
model_data = pd.concat([model_data['Churn?_True.'], model_data.drop(['Churn?_False.', 'Churn?_True.'], axis=1)],
                       axis=1)
model_data

将数据拆分为训练、验证和测试集。这将有助于防止模型的过拟合和对未看到的数据测试模型的准确性。

In [ ]:
train_data, validation_data, test_data = np.split(model_data.sample(frac=1, random_state=1729),
                                                  [int(0.7 * len(model_data)), int(0.9 * len(model_data))])
train_data.to_csv('train.csv', header=False, index=False)
validation_data.to_csv('validation.csv', header=False, index=False)

把文件上传到 S3 里。

In [ ]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

---
## 模型训练

开始进行模型训练，指定 XGBoost 算法容器的路径。

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'xgboost')

然后，因为使用 CSV 文件格式进行训练，所以需要创建 s3_input 并且在训练函数中指向 S3 中文件。

In [ ]:
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

然后指定 XGBoost 用到的一些超参数，举例如下：

- `max_depth` 控制算法中每个树的构建深度。更深的树可以更好地拟合，但计算费用更高，并可能导致过度拟合。通常需要权衡大量少层树和少量深层树对模型性能的影响。
- `subsample` 控制训练数据的采样。此超参数有助于减少过度拟合，但将其设置得太低也会导致数据模型耗尽。
- `num_round` 控制提升回合的数量。此值指定随后使用先前迭代的残差训练的模型。同样，更多的回合应该能够更好地拟合训练数据，但是计算成本昂贵或导致过度拟合。
- `eta` 控制每一轮提升的积极性。较大的值会导致更保守的提升。
- `gamma` 控制树生长的密集程度。值越大，模型就越保守。

In [ ]:
sess = sagemaker.Session()

# Use Spot instance for cost optimization
xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m5.large',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    train_use_spot_instances=True,
                                    train_max_wait=3600,
                                    train_max_run=1800,
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation}) 

---
## 模型编译
[Amazon SageMaker Neo](https://aws.amazon.com/sagemaker/neo/) 
- 可以对模型进行优化，使其运行速度高达两倍，同时不会损失精度。
- 使用 `compile_model()` 功能，指定目标实例类型（比如m5）以及存储编译模型的 S3 存储桶

In [ ]:
compiled_model = xgb
try:
    xgb.create_model()._neo_image_account(boto3.Session().region_name)
except:
    print('Neo is not currently supported in', boto3.Session().region_name)
else:
    output_path = '/'.join(xgb.output_path.split('/')[:-1])
    compiled_model = xgb.compile_model(target_instance_family='ml_m5',
                                       input_shape={'data':[1, 69]},
                                       role=role,
                                       framework='xgboost',
                                       framework_version='0.7',
                                       output_path=output_path)
    compiled_model.name = 'deployed-demo-xgboost-customer-churn-' + current_time
    compiled_model.image = get_image_uri(sess.boto_region_name, 'xgboost-neo', repo_version='latest')

---
## 模型托管

通过使用指定好的算法，现在创建一个模型并部署到托管终端节点。

In [ ]:
xgb_predictor = compiled_model.deploy(initial_instance_count = 1,
                                      instance_type = 'ml.m5.large')

## 模型评估

运行托管终端节点后，可以非常方便地从模型中进行实时预测，只需发出 HTTP POST 请求即可。
设置序列化和反序列化来将 `test_data` NumPy 数组传递给终端节点后面的模型。

In [ ]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer
xgb_predictor.deserializer = None

使用一个简单的功能进行：
1. 遍历测试数据集
2. 将其拆分小批次的行
3. 将这些小批次行转换为 CSV 字符串有效负载
4. 通过调用 XGBoost 终端节点获取小批次的预测
5. 收集预测并从模型提供的 CSV 输出转换为 NumPy 数组

In [ ]:
def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(test_data.to_numpy()[:, 1:])

有很多方法可以比较机器学习模型的性能，首先先简单地将实际值与预测值进行比较。
在这个 Demo 中，只是简单地预测客户是否流失（`1`或者`0`），它会产生一个简单的混淆矩阵

In [ ]:
pd.crosstab(index=test_data.iloc[:, 0], columns=np.round(predictions), rownames=['actual'], colnames=['predictions'])

_由于算法的随机元素，结果可能会略有不同。_

在这些流失者中:
- 绝大部分我们做了正确地预测（True Positive & True Negative）
- 有个别错误地预测了客户会流失，然后最终并没有流失（False Negative）
- 还有个别预测了客户不会流失，然后最终却流失了 (False Positive)

由于上面的 `np.round()` 函数使用的是一个简单的阈值（临界值）0.5。
从 `xgboost` 的预测出现了0和1之间的连续值，使得它们变成二分类的状况。
但是，由于客户流失的预计会给公司带来更多的费用，而不是主动地试图留住我们认为可能流失的客户，因此我们应考虑调整此阈值(临界值)。
这几乎肯定会增加误报的数量，但也可以预期会增加真的正面预测的数量，减少误报的数量。

为了在这里得到一个粗略的直觉，接下来看看预测的连续值。

In [ ]:
plt.hist(predictions)
plt.show()

来自模型的连续值预测倾向于向 0 或 1 倾斜，但在 0.1 到 0.9 之间有足够多的取值。
用来调整临界值确实会改变一些客户的预测。例如...

In [ ]:
pd.crosstab(index=test_data.iloc[:, 0], columns=np.where(predictions > 0.3, 1, 0))

我们可以看到，将临界值从 0.5 更改为 0.3 会导致预测结果出现少量变化。
这里的数字总体来说很少，但是由于临界值的变化，总体上只有 6-10% 的客户正在发生变化。
这是正确的决定吗？我们最终可能会留住个别额外的客户，但我们也额外不必要地激励了本来会留下的客户。
确定最佳临界点是在真实环境中正确应用机器学习的关键步骤。
比如，寻求最佳的临界值以获得更高的投资回报率。

## （可选）清理环境

如果已经完成了 Demo，可以运行下面的命令来删除创建的托管终端节点，并避免留下的实例产生的任何费用。

In [ ]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)